# SageMaker distributed data parallelism (SMDDP)

```python
# configuration for running training on smdistributed Data Parallel
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

# git configuration to download our fine-tuning script
git_config = {'repo': 'https://github.com/huggingface/transformers.git','branch': 'v4.26.0'}

instance_type='ml.p3.16xlarge'
instance_count=2
volume_size=200

huggingface_estimator = HuggingFace(entry_point='run_qa.py',
                                    source_dir='./examples/pytorch/question-answering',
                                    git_config=git_config,
                                    instance_type=instance_type,
                                    instance_count=instance_count,
                                    volume_size=volume_size,
                                    distribution= distribution)
```

In [2]:
!pip install "sagemaker>=2.140.0" "transformers==4.26.1" "datasets[s3]==2.10.1" "torch" --upgrade


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import sagemaker.huggingface

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


## Permissions

In [4]:
# ACCESS to an IAM role

import sagemaker
import boto3
sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker role arn: arn:aws:iam::802575742115:role/service-role/AmazonSageMaker-ExecutionRole-20230929T143152
sagemaker bucket: sagemaker-us-east-1-802575742115
sagemaker session region: us-east-1


## Load Dataset

In [5]:
from datasets import load_dataset, VerificationMode
from transformers import AutoTokenizer

# tokenizer used in preprocessing
tokenizer_name = 'distilbert-base-uncased'

# download tokenizer
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

# tokenizer helper function
def tokenize(batch):
    return tokenizer(batch['text'], padding='max_length', truncation=True)

# load dataset
train_dataset, test_dataset = load_dataset('imdb', split=['train', 'test'], verification_mode=VerificationMode.NO_CHECKS)
test_dataset = test_dataset.shuffle().select(range(10000)) # smaller the size for test dataset to 10k 

print(train_dataset[0:2])

# tokenize dataset
train_dataset = train_dataset.map(tokenize, batched=True)
test_dataset = test_dataset.map(tokenize, batched=True)

print(train_dataset[0:2])

# input_ids are tokenized and mapped to corresponding indices in the model's vocabulary.

# segment_ids 
# BERT was designed to handle tasks involving multiple sentences. segment_ids are used to distinguish different sentences in the input.
# For a single sentence task, all elements in segment_ids would be set to 0. For tasks involving multiple sentences, you might have alternating 0s and 1s to indicate different segments.

# attention_mask:
# BERT models have a fixed input size, the attention mask is used to indicate which elements of the input sequence are actual tokens versus padding tokens.
# 0 indicates it is a padding token. This mask helps the model focus only on the relevant tokens and ignore the padding tokens during training.

# set format for pytorch
# set_format('torch'): This method is used to set the format of the dataset to be compatible with PyTorch. 
# columns=['input_ids', 'attention_mask', 'labels']: This specifies which columns of the dataset should be converted to PyTorch tensors. 
train_dataset =  train_dataset.rename_column("label", "labels")
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset = test_dataset.rename_column("label", "labels")
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'labels'])

print(train_dataset[0:2])

Found cached dataset parquet (/root/.cache/huggingface/datasets/parquet/plain_text-745310791ff4d097/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /root/.cache/huggingface/datasets/parquet/plain_text-745310791ff4d097/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-ec473c0cf9d9f4ca.arrow


{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

{'text': ['I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far b

## Uploading data to sagemaker_session_bucket

In [6]:
# s3 key prefix for the data
s3_prefix = 'samples/datasets/imdb'

# save train_dataset to s3
training_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/train'
train_dataset.save_to_disk(training_input_path)

# save test_dataset to s3
test_input_path = f's3://{sess.default_bucket()}/{s3_prefix}/test'
test_dataset.save_to_disk(test_input_path)

Saving the dataset (0/1 shards):   0%|          | 0/25000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

## Fine-tuning & starting SageMaker Training Job
In order to create a sagemaker training job we need an HuggingFace Estimator. The Estimator handles end-to-end Amazon SageMaker training and deployment tasks. In a Estimator we define, which fine-tuning script should be used as entry_point, which instance_type should be used, which hyperparameters are passed in .....

```python
huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            base_job_name='huggingface-sdk-extension',
                            instance_type='ml.p3.2xlarge',
                            instance_count=1,
                            transformers_version='4.4',
                            pytorch_version='1.6',
                            py_version='py36',
                            role=role,
                            hyperparameters = {'epochs': 1,
                                               'train_batch_size': 32,
                                               'model_name':'distilbert-base-uncased'
                                                })
```

When we create a SageMaker training job, SageMaker takes care of starting and managing all the required ec2 instances for us with the huggingface container, uploads the provided fine-tuning script train.py and downloads the data from our sagemaker_session_bucket into the container at /opt/ml/input/data. Then, it starts the training job by running.

```python
/opt/conda/bin/python train.py --epochs 1 --model_name distilbert-base-uncased --train_batch_size 32
The hyperparameters you define in the HuggingFace estimator are passed in as named arguments.
```

Sagemaker is providing useful properties about the training environment through various environment variables, including the following:

- SM_MODEL_DIR: A string that represents the path where the training job writes the model artifacts to. After training, artifacts in this directory are uploaded to S3 for model hosting.

- SM_NUM_GPUS: An integer representing the number of GPUs available to the host.

- SM_CHANNEL_XXXX: A string that represents the path to the directory that contains the input data for the specified channel. For example, if you specify two input channels in the HuggingFace estimator’s fit call, named train and test, the environment variables SM_CHANNEL_TRAIN and SM_CHANNEL_TEST are set.

To run your training job locally you can define instance_type='local' or instance_type='local_gpu' for gpu usage. Note: this does not working within SageMaker Studio

## Creating an Estimator and start a training job

In [8]:
from sagemaker.huggingface import HuggingFace

# hyperparameters, which are passed into the training job
hyperparameters={'epochs': 1,
                 'train_batch_size': 32,
                 'model_name':'distilbert-base-uncased'
                 }

# configuration for running training on smdistributed Data Parallel
distribution = {'smdistributed':{'dataparallel':{ 'enabled': True }}}

# instance configurations
instance_type='ml.p3.16xlarge'
instance_count=2
volume_size=200

huggingface_estimator = HuggingFace(entry_point='train.py',
                            source_dir='./scripts',
                            instance_type=instance_type,
                            instance_count=instance_count,
                            volume_size=volume_size,
                            role=role,
                            transformers_version='4.26.0',
                            pytorch_version='1.13.1',
                            py_version='py39',
                            distribution= distribution,
                            hyperparameters = hyperparameters)

# starting the train job with our uploaded datasets as input
huggingface_estimator.fit({'train': training_input_path, 'test': test_input_path})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2024-01-23-04-10-36-884


2024-01-23 04:10:37 Starting - Starting the training job...
2024-01-23 04:10:49 Pending - Training job waiting for capacity......
2024-01-23 04:11:55 Pending - Preparing the instances for training.........
2024-01-23 04:13:36 Downloading - Downloading the training image...........................
2024-01-23 04:17:47 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
2024-01-23 04:18:10,761 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2024-01-23 04:18:10,822 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-01-23 04:18:10,834 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
2024-01-23 04:18:10,836 sagemaker_pytorch_container.training INFO     Invoking SMDataParallel
2024-01-23 04:18:10,837 sagemaker_pytorch_container.training INFO  

## Deploying the endpoint

To deploy our endpoint, we call deploy() on our HuggingFace estimator object, passing in our desired number of instances and instance type.

In [10]:
predictor = huggingface_estimator.deploy(1, "ml.g4dn.xlarge")

sentiment_input= {"inputs":"I am not sure whether I will use the new Inference DLC."}
predictor.predict(sentiment_input)

INFO:sagemaker:Creating model with name: huggingface-pytorch-training-2024-01-23-04-21-48-811
INFO:sagemaker:Creating endpoint-config with name huggingface-pytorch-training-2024-01-23-04-21-48-811
INFO:sagemaker:Creating endpoint with name huggingface-pytorch-training-2024-01-23-04-21-48-811


--------!

[{'label': 'LABEL_0', 'score': 0.5309359431266785}]

In [ ]:
sentiment_input= {"inputs":"I love using the new Inference DLC."}
predictor.predict(sentiment_input)

In [ ]:
sentiment_input= {"inputs":"Caring is most important"}
predictor.predict(sentiment_input)

In [ ]:
# Delete the endpoint
predictor.delete_model()
predictor.delete_endpoint()

## Estimator Parameters

In [ ]:
# container image used for training job
print(f"container image used for training job: \n{huggingface_estimator.image_uri}\n")

# s3 uri where the trained model is located
print(f"s3 uri where the trained model is located: \n{huggingface_estimator.model_data}\n")

# latest training job name for this estimator
print(f"latest training job name for this estimator: \n{huggingface_estimator.latest_training_job.name}\n")

# access the logs of the training job
huggingface_estimator.sagemaker_session.logs_for_job(huggingface_estimator.latest_training_job.name)

In [ ]:
predictor.delete_endpoint()

## Upload the fine-tuned model to huggingface.co